In [0]:
rfm_metrics_df=spark.read.format('delta').load('abfss://data@storagecustomer260.dfs.core.windows.net/catalog/silver/customer_classification_segmentation/data')
support_ticket_metrics_df=spark.read.format('delta').load('abfss://data@storagecustomer260.dfs.core.windows.net/catalog/silver/support_ticket_metrics/data')
web_activity_metric_df=spark.read.format('delta').load('abfss://data@storagecustomer260.dfs.core.windows.net/catalog/silver/web_activity_metrics/data')
customer_master_df=spark.read.format('delta').load('abfss://data@storagecustomer260.dfs.core.windows.net/catalog/silver/customer_master_data/data')

In [0]:
customer_master_combined_df=customer_master_df.join(rfm_metrics_df,customer_master_df.customer_id==rfm_metrics_df.customer_id,'left').drop(rfm_metrics_df.customer_id)
customer_master_combined_df=customer_master_combined_df.join(support_ticket_metrics_df,customer_master_combined_df.customer_id==support_ticket_metrics_df.customer_id,'left').drop(support_ticket_metrics_df.customer_id)
customer_master_combined_df=customer_master_combined_df.join(web_activity_metric_df,customer_master_combined_df.customer_id==web_activity_metric_df.customer_id,'left').drop(web_activity_metric_df.customer_id)

In [0]:
from pyspark.sql.functions import *
customer_master_combined_df=customer_master_combined_df.withColumn('customer_category',when((col('customer_type')=='Champions')&(col('engagement_level')=='Highly Engaged')&(col('segmentation')=='Low complaints'),'VIP')\
    .otherwise(when((col('customer_type')=='Loyal_customers')&(col('engagement_level')=='Browsers')&(col('segmentation')=='Moderate Complaints'),'Loyal But Low Engagement')\
        .otherwise(when((col('customer_type')=='At Risk')&(col('engagement_level')=='Silent Leavers'),'At Risk')\
            .otherwise(when((col('customer_type')=='Lost')&(col('engagement_level')=='Dormant Users'),'Lost Customers')\
                .otherwise(when((col('customer_type')=='Big Spenders') & (col('engagement_level')=='Highly Engaged'), 'High Value Opportunity')\
                    .otherwise(when(col('segmentation')=='Pending Risk', 'Service Risk Customers')\
                        .otherwise(when((col('customer_type')=='Loyal_customers') & (col('engagement_level')=='Highly Engaged'), 'Brand Advocates')\
                            .otherwise('Others')
                            ))
                    ))
        )
                                                                   
)).withColumn('updated_time',current_timestamp())
display(customer_master_combined_df)
customer_master_combined_df.write.format('csv').mode('overwrite').option('mergeSchema',True).option('header',True).save('abfss://data@storagecustomer260.dfs.core.windows.net/catalog/gold/customer_360_final_view/data')

customer_id,first_name,last_name,email,phone,address,frequency,monetary,recency,churn_status,classification_timestamp,customer_type,total_closed_tickets,avg_resolution_time,total_open_tickets,total_ticket_count,segmentation,latest_login_date,average_duration_time,total_pages_visited,days_since_last_login,engagement_level,customer_category,updated_time
101,John,Doe,user1@mail.com,9000000001,Pune,41,111714.31999999999,0,Active,2025-11-01T18:53:18.511Z,Champions,1,3.0,1,2,Moderate Complaints,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
103,Priya,Singh,user3@mail.com,9000000003,Mumbai,39,116722.24000000002,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,2024-01-02,15.0,19,669,Dormant Users,Others,2025-11-02T06:46:17.056Z
104,Sara,Khan,user4@mail.com,9000000004,Delhi,34,84406.79999999999,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,2025-05-31,14.0,18,154,Dormant Users,Others,2025-11-02T06:46:17.056Z
105,Mike,Smith,user5@mail.com,9000000005,Delhi,36,88914.38,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
106,Nina,Patel,user6@mail.com,9000000006,Hyderabad,36,88556.17000000001,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
107,Ravi,Verma,user7@mail.com,9000000007,Delhi,32,83757.01999999999,0,Active,2025-11-01T18:53:18.511Z,Champions,1,4.0,null,1,Low Complaints,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
109,Yash,Gupta,user9@mail.com,9000000009,Bangalore,27,73041.05000000002,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
110,Emma,Jones,user10@mail.com,9000000010,Delhi,23,58188.65999999999,0,Active,2025-11-01T18:53:18.511Z,Champions,2,6.5,null,2,Moderate Complaints,2024-03-02,20.0,13,609,Dormant Users,Others,2025-11-02T06:46:17.056Z
111,Ali,Hasan,user11@mail.com,9000000011,Pune,30,95312.48000000001,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,2025-05-11,6.0,13,174,Dormant Users,Others,2025-11-02T06:46:17.056Z
112,Karan,Gill,user12@mail.com,9000000012,Pune,31,70950.51000000001,0,Active,2025-11-01T18:53:18.511Z,Champions,null,null,null,null,null,null,null,null,null,null,Others,2025-11-02T06:46:17.056Z
